Распаковка выборки

In [22]:
!unzip limb.zip
!unzip perlimb.zip

Archive:  limb.zip
replace limb/convert.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Xception perelimb начальное обучение

In [8]:
import tensorflow as tf
from keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "perelimb/train"  # путь для обучающей выборки
test_path = "perelimb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
rescale=1.0/255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 16
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель Xception
xception = Xception(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="limb_model2.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх Xception
x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
x = Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)

# Создайте модель на базе Xception
model = Model(inputs=xception.input, outputs=output)

# Заморозьте веса предобученных слоев
for layer in xception.layers:
 layer.trainable = False

# Скомпилируйте и обучите модель
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат = ", accuracy[1])



Found 190 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Epoch 1/15
12/12 [==============================] - 18s 1s/step - loss: 0.7963 - accuracy: 0.5421 - val_loss: 0.6246 - val_accuracy: 0.6377
Epoch 2/15
12/12 [==============================] - 14s 1s/step - loss: 0.7033 - accuracy: 0.5632 - val_loss: 0.6054 - val_accuracy: 0.6377
Epoch 3/15
12/12 [==============================] - 14s 1s/step - loss: 0.7034 - accuracy: 0.6368 - val_loss: 0.5829 - val_accuracy: 0.6667
Epoch 4/15
12/12 [==============================] - 16s 1s/step - loss: 0.7440 - accuracy: 0.5737 - val_loss: 0.6348 - val_accuracy: 0.6377
Epoch 5/15
12/12 [==============================] - 14s 1s/step - loss: 0.6527 - accuracy: 0.6421 - val_loss: 0.5666 - val_accuracy: 0.7101
Epoch 6/15
12/12 [==============================] - 14s 1s/step - loss: 0.6896 - accuracy: 0.6000 - val_loss: 0.5701 - val_accuracy: 0.7101
Epoch 7/15
12/12 [==============================] - 14s 1s/step - loss: 0.6024 

### Xception perelimb fine tuning




In [10]:
from keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint


train_path = "train"  # путь для обучающей выборки
test_path = "test" # путь для контрольной выборки

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
) 

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499 # размерность тензора на основе изображения для входных данных в нейронную сеть
batch_size = 8 # размерность подвыборки
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
) # генератор для обучения 

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
) # генератор для тестирования на контрольной выборке

# загрузка модели Xception
xception = Xception(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="xception_limb_model3.h5", monitor="val_accuracy", save_best_only=True)

x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = Dropout(0.5)(x)  # добавление dropout для предотвращения переобучения
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)  # добавление dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)


# создание модели на базе Xception
model = Model(inputs=xception.input, outputs=output)
model.load_weights("limb_Xception.h5") # загрузка весов предобученной модели для операции fine tuning

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])


Found 190 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 499, 499, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 249, 249, 32)         864       ['input_7[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 249, 249, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_con

### Xception perelimb финальная прогонка

In [14]:
from os import listdir

import matplotlib.pyplot as plt
import keras.utils as image
from keras.models import load_model


def result(
    dir, true, pred1, pred2, model
):  # функция, выводящая результат распознавания каждого изображения в каждой папке контрольной выборки
    count_bolen = 0
    count_zdorov = 0
    img_dir = listdir(dir)
    for file in img_dir:  # перебор файлов в папке
        img = image.load_img(
            dir + "/" + file, target_size=(499, 499)
        )  # загрузка изображения
        x = image.img_to_array(img)
        x = x.reshape(1, 499, 499, 3)
        x /= 255
        prediction = model.predict(x)  # обработка нейросетью

        if prediction[0][0] < (
            0.5
        ):  # вывод результата в зависимости от успешности распознавания
            if true == "zdorov":
                count_zdorov += 1
                # счётчик ошибок "здоровых" (якобы больных)
                show(pred1, prediction[0][0], img)  # болен
        else:
            if true == "bolen":
                count_bolen += 1  # счётчик ошибок "больных" (якобы здоровых)
                show(pred2, prediction[0][0], img)  # здоров
            show(pred2, prediction[0][0], img)

    print("Якобы здоровых: ", count_bolen)
    print("Якобы больных: ", count_zdorov)


def show(title, pred, img):  # функция показа результата распознавания
    plt.imshow(img.convert("RGBA"))
    plt.title(title + str(pred))
    plt.show()


model = load_model("xception_perelimb.keras")

result("perelimb/test/bolen", "bolen", "Патология -> ", "Якобы условная норма -> ", model)
result("perelimb/test/zdorov", "zdorov", "Якобы патология -> ", "Условная норма -> ", model)


Output hidden; open in https://colab.research.google.com to view.

### Xception limb начальное обучение




In [16]:
import tensorflow as tf
from keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "limb/train"  # путь для обучающей выборки
test_path = "limb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
rescale=1.0/255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 16
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель Xception
xception = Xception(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="XCEPTION_limb_model.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх Xception
x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
x = Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)

# Создайте модель на базе Xception
model = Model(inputs=xception.input, outputs=output)

# Заморозьте веса предобученных слоев
for layer in xception.layers:
 layer.trainable = False

# Скомпилируйте и обучите модель
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат = ", accuracy[1])



Found 339 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/15
22/22 [==============================] - 30s 1s/step - loss: 0.7375 - accuracy: 0.5723 - val_loss: 0.5545 - val_accuracy: 0.7250
Epoch 2/15
22/22 [==============================] - 26s 1s/step - loss: 0.7235 - accuracy: 0.5959 - val_loss: 0.6636 - val_accuracy: 0.5833
Epoch 3/15
22/22 [==============================] - 25s 1s/step - loss: 0.6819 - accuracy: 0.6342 - val_loss: 0.5727 - val_accuracy: 0.7667
Epoch 4/15
22/22 [==============================] - 25s 1s/step - loss: 0.6275 - accuracy: 0.6726 - val_loss: 0.5414 - val_accuracy: 0.7583
Epoch 5/15
22/22 [==============================] - 24s 1s/step - loss: 0.6087 - accuracy: 0.6578 - val_loss: 0.5083 - val_accuracy: 0.7250
Epoch 6/15
22/22 [==============================] - 30s 1s/step - loss: 0.6212 - accuracy: 0.6903 - val_loss: 0.4721 - val_accuracy: 0.7417
Epoch 7/15
22/22 [==============================] - 25s 1s/step - loss: 0.5731

### Xception limb fine tuning




In [19]:
import tensorflow as tf
from keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Задайте размер изображений

train_path = "limb/train"  # путь для обучающей выборки
test_path = "limb/test"
# Создайте генераторы данных для обучения и проверки
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

img_width, img_height = 499, 499
batch_size = 8
train_generator = train_datagen.flow_from_directory(
train_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
test_path,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode="binary"
)

# Загрузите предобученную модель Xception
xception = Xception(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
callback = ModelCheckpoint(filepath="XCEPTION_limb_model_tuned.h5", monitor="val_accuracy", save_best_only=True)
# Добавьте свои слои поверх Xception
x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(516, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
x = Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)  # добавим dropout для предотвращения переобучения
output = Dense(1, activation="sigmoid")(x)


# Создайте модель на базе Xception
model = Model(inputs=xception.input, outputs=output)
model.load_weights("XCEPTION_limb_model.h5")

model.summary()
model.compile(optimizer=Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_generator, epochs=15, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator),callbacks=[callback])

accuracy = model.evaluate(test_generator)

print("Результат fine tune'd = ", accuracy[1])
model.save("xception_limb.keras")

Found 339 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 499, 499, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 249, 249, 32)         864       ['input_10[0][0]']            
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 249, 249, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_co

### Xception limb финальная прогонка




In [21]:
from os import listdir

import matplotlib.pyplot as plt
import keras.utils as image
from keras.models import load_model


def result(
    dir, true, pred1, pred2, model
):  # функция, выводящая результат распознавания каждого изображения в каждой папке контрольной выборки
    count_bolen = 0
    count_zdorov = 0
    img_dir = listdir(dir)
    for file in img_dir:  # перебор файлов в папке
        img = image.load_img(
            dir + "/" + file, target_size=(499, 499)
        )  # загрузка изображения
        x = image.img_to_array(img)
        x = x.reshape(1, 499, 499, 3)
        x /= 255
        prediction = model.predict(x)  # обработка нейросетью

        if prediction[0][0] < (
            0.5
        ):  # вывод результата в зависимости от успешности распознавания
            if true == "zdorov":
                count_zdorov += 1
                # счётчик ошибок "здоровых" (якобы больных)
                show(pred1, prediction[0][0], img)  # болен
        else:
            if true == "bolen":
                count_bolen += 1  # счётчик ошибок "больных" (якобы здоровых)
                show(pred2, prediction[0][0], img)  # здоров
            show(pred2, prediction[0][0], img)

    print("Якобы здоровых: ", count_bolen)
    print("Якобы больных: ", count_zdorov)


def show(title, pred, img):  # функция показа результата распознавания
    plt.imshow(img.convert("RGBA"))
    plt.title(title + str(pred))
    plt.show()


model = load_model("xception_limb.keras")

result("limb/test/bolen", "bolen", "Патология -> ", "Якобы условная норма -> ", model)
result("limb/test/zdorov", "zdorov", "Якобы патология -> ", "Условная норма -> ", model)


Output hidden; open in https://colab.research.google.com to view.